In [5]:
import numpy as np
#Sahil
data_path = 'dataexm.npz'

npzdata = np.load(data_path)
dataset = npzdata['dataset']
print(data.shape)

(2494, 21412)


In [3]:
#Loading important libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
#Removing events which are not needed for current task
dataset = dataset[dataset[:, -1]!=9]
dataset = dataset[dataset[:, -1]!=10]

dataset = dataset[dataset[:, -1]!=3]
dataset = dataset[dataset[:, -1]!=6]
dataset = dataset[dataset[:, -1]!=7]
dataset = dataset[dataset[:, -1]!=8]
dataset = dataset[dataset[:, -1]!=11]

print(dataset.shape)

#Defining x and y
x = dataset[:, :-1]
y = dataset[:, -1]
print(x.shape)
print(y.shape)
print(list(set(y)))

permutations = np.random.permutation(len(x))
# print(permutation)
print(x[-1])
x = x[permutations]
y = y[permutations]
print(x[-1])

xmlp = x.copy()
ymlp = y.copy()

event_shape = (351, 61)

xlstm = x.copy().reshape((-1, event_shape[0], event_shape[1]))
ylstm = y.copy()

xcnn = x.copy().reshape((-1, event_shape[0], event_shape[1]))
ycnn = y.copy()

(2397, 21412)
(2397, 21411)
(2397,)
[1.0, 2.0, 4.0, 5.0]
[-3.35998300e-06 -2.95636516e-06 -3.45496355e-06 ...  2.48586036e-06
  2.51273527e-06  2.85969160e-06]
[1.34827604e-06 9.70851669e-07 1.93598707e-06 ... 1.42627650e-06
 1.02899538e-06 1.03489261e-06]


In [12]:
#Encoding y
dict = {1.0: 0,2.0: 1, 4.0: 2, 5.0: 3}

for i in range(len(y)):
    y[i] = dict[y[i]]
    ylstm[i] = dict[ylstm[i]]

print(list(set(y)))

[0.0, 1.0, 2.0, 3.0]


In [13]:
num_classes = 4
y = tf.keras.utils.to_categorical(y, num_classes)
ymlp = y.copy()
ylstm = y.copy()
ycnn = y.copy()
# yab10_1[0]
print(y[:5])

[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [14]:
train_test_split = int(0.8 * len(xcnn))
print(train_test_split)

x_train_cnn = xcnn[:train_test_split]
y_train_cnn = ycnn[:train_test_split]

x_test_cnn = xcnn[train_test_split:]
y_test_cnn = ycnn[train_test_split:]

1917


In [15]:
def get_cyclic_generator(features, labels, batch_size = 1):
  while True:
    for n in range(features.shape[0]//batch_size):
      yield (features[n*batch_size: (n+1)*batch_size], labels[n*batch_size: (n+1)*batch_size])
    permuted = np.random.permutation(len(features))
    features = features[permuted]
    labels = labels[permuted]

In [16]:
train_batch_size = 32
train_cyclic_generator_cnn = get_cyclic_generator(x_train_cnn, y_train_cnn, batch_size = train_batch_size)
# test_cyclic_generator = get_cyclic_generator(x_test, y_test, batch_size = train_batch_size)

In [18]:
inp_shape = event_shape
ip = tf.keras.Input(shape = inp_shape)
conv1 = tf.keras.layers.Conv1D(32, 3,)(ip)
rel = tf.keras.layers.LeakyReLU()(conv1)
conv2 = tf.keras.layers.Conv1D(16, 3,)(rel)
rel2 = tf.keras.layers.LeakyReLU()(conv2)
# lstm = tf.keras.layers.LSTM(16, return_sequences=True)(ip)
flatten = tf.keras.layers.Flatten()(rel2)
dense1 = tf.keras.layers.Dense(units = 32, kernel_initializer = 'random_normal')(flatten)
leakyRelu = tf.keras.layers.LeakyReLU()(dense1)
drop1 = tf.keras.layers.Dropout(.4)(leakyRelu)
dense2 = tf.keras.layers.Dense(units = 16, kernel_initializer = 'random_normal')(drop1)
leakyRelu2 = tf.keras.layers.LeakyReLU()(dense2)
drop2 = tf.keras.layers.Dropout(.4)(dense2)
out = tf.keras.layers.Dense(num_classes, activation = 'softmax')(drop2)

cnn = tf.keras.Model(inputs = ip, outputs = out)
cnn.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 351, 61)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 349, 32)           5888      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 349, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 347, 16)           1552      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 347, 16)           0         
_________________________________________________________________
flatten (Flatten)            (None, 5552)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                177696

In [19]:
# Compiling the model
cnn_earlystop = EarlyStopping(patience = 10, min_delta = 0.001, monitor = 'val_acc')

path = 'cnn_checkpoint/checkpoint_{epoch:02d}';
cnn_checkpoint = ModelCheckpoint(filepath = path,
                            verbose = 1,
                            monitor = 'val_acc',
                            save_freq = 'epoch',
                            save_best_only = True,
                            save_weights_only = True)

cnn.compile(loss = 'categorical_crossentropy', metrics = ['acc'], optimizer = 'adam')

In [20]:
cnn_history = cnn.fit(train_cyclic_generator_cnn,
                        validation_data = (x_test_cnn, y_test_cnn),
                        steps_per_epoch= 449,
                        epochs = 50,
                        callbacks = [cnn_earlystop, cnn_checkpoint]) #Fitting the model

Epoch 1/50
449/449 [==============================] - 14s 22ms/step - loss: 1.2793 - acc: 0.3793 - val_loss: 1.2627 - val_acc: 0.4125

Epoch 00001: val_acc improved from -inf to 0.41250, saving model to cnn_checkpoint\checkpoint_01
Epoch 2/50
449/449 [==============================] - 9s 20ms/step - loss: 1.2592 - acc: 0.3946 - val_loss: 1.2659 - val_acc: 0.4125

Epoch 00002: val_acc did not improve from 0.41250
Epoch 3/50
449/449 [==============================] - 10s 21ms/step - loss: 1.2556 - acc: 0.3978 - val_loss: 1.2657 - val_acc: 0.4125

Epoch 00003: val_acc did not improve from 0.41250
Epoch 4/50
449/449 [==============================] - 10s 21ms/step - loss: 1.2512 - acc: 0.4009 - val_loss: 1.2686 - val_acc: 0.4125

Epoch 00004: val_acc did not improve from 0.41250
Epoch 5/50
449/449 [==============================] - 10s 22ms/step - loss: 1.2539 - acc: 0.3934 - val_loss: 1.2651 - val_acc: 0.4125

Epoch 00005: val_acc did not improve from 0.41250
Epoch 6/50
449/449 [=========

KeyboardInterrupt: 

In [ ]:
#Plotting model history
plt.plot(cnn_history.history['acc'])
plt.plot(cnn_history.history['loss'])
plt.legend(['Training Accuracy', 'Training Loss'])
plt.title('Training History')
plt.ylabel('Value')
plt.xlabel('epochs')
plt.show()
print("Initial training accuracy is: ", cnn_history.history['acc'][0])
print("Final training accuracy is: ", cnn_history.history['acc'][-1])

print("Initial training loss is: ", cnn_history.history['loss'][0])
print("Final training loss is: ", cnn_history.history['loss'][-1])

In [ ]:
#Plotting model history
plt.plot(cnn_history.history['acc'])
plt.plot(cnn_history.history['loss'])
plt.legend(['Training Accuracy', 'Training Loss'])
plt.title('Training History')
plt.ylabel('Value')
plt.xlabel('epochs')
plt.show()
print("Initial training accuracy is: ", cnn_history.history['acc'][0])
print("Final training accuracy is: ", cnn_history.history['acc'][-1])

print("Initial training loss is: ", cnn_history.history['loss'][0])
print("Final training loss is: ", cnn_history.history['loss'][-1])